## Chi square test

In the previous project, we identified a subset of applicants who don't complete the admissions quiz. Then we developed a null and alternative hypothesis that we want to test in an experiment.

In this project, we'll conduct our experiment. First, we'll determine how long we need to run our experiment in order to detect a significant difference between our control and treatment groups. Then we'll run our experiment and evaluate our results using a chi-square test. 

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import scipy

#from pymongo import MongoClient
from statsmodels.stats.contingency_tables import Table2x2
from statsmodels.stats.power import GofChisquarePower
from teaching_tools.ab_test.experiment import Experiment
from teaching_tools.ab_test.reset import Reset

r = Reset()
r.reset_database()

In [ ]:
client = MongoClient(host="localhost", port=27017)
db = client["wqu-abtest"]
ds_app = db["ds-applicants"]
print("client:", type(client))
print("ds_app:", type(ds_app))

## Calculate Power

One of a Data Scientist's jobs is to help others determine what's  meaningful information and what's not. You can think about this as distinguishing between **signal** and **noise**. As the author [Nate Silver](https://en.wikipedia.org/wiki/The_Signal_and_the_Noise) puts it, "The signal is the truth. The noise is what distracts us from the truth."

In our experiment, we're looking for a signal indicating that applicants who receive an email are more likely to complete the admissions quiz. If signal's strong, it'll be easy to see. A much higher number of applicants in our treatment group will complete the quiz. But if the signal's weak and there's only a tiny change in quiz completion, it will be harder to determine if this is a meaningful difference or just random variation. How can we separate signal from noise in this case? The answer is **statistical power**.

To understand what statistical power is, let's imagine that we're radio engineers building an antenna. The size of our antenna would depend on the type of signal we wanted to detect. It would be OK to build a low-power antenna if we only wanted to detect strong signals, like a car antenna that picks up your favorite local music station. But our antenna wouldn't pick up weaker signals — like a radio station on the other side of the globe. For weaker signals, we'd need something with higher power. In statistics, power comes from the number of observations you include in your experiment. In other words, the more people we include, the stronger our antenna, and the better we can detect weak signals. 

To determine exactly how many people we should include in our study, we need to do a **power calculation**. 

First, instantiate a `GofChisquarePower` object and assign it to the variable name `chi_square_power`. Then use it to calculate the `group_size` needed to detect an effect size of `0.2`, with an alpha of `0.05` and power of `0.8`.

In [ ]:
chi_square_power = GofChisquarePower()
group_size = math.ceil(chi_square_power.solve_power(effect_size=0.2,alpha=0.5,power=0.8)) #ceil 196.33 = 197 floor 196.33 = 196
print("Group size:", group_size)
print("Total # of applicants needed:", group_size * 2)

The results here are telling us that if we want to detect an effect size of 0.2 we need a group size of about 200 people. Since our experiment has two conditions (treatment and control, or email and no email), that means we need a total of about 400 applicants in our experiment.

But what about detecting other effect sizes? If we needed to detect a *larger* effect size, we'd need *fewer* applicants. If we needed to detect a *smaller* effect size, we'd need *more* applicants. One way to visualize the relationship between effect size, statistical power, and number of applicants is to make a graph.

Use `chi_square_power` to plot a power curve for three effect sizes: `0.2`, `0.5`, and `0.8`. The x-axis should be the number of observations, ranging from `0` to twice the `group_size` from the previous task. 

In [ ]:
n_observations = np.arange(0, group_size * 2 + 1) ##we need the value in np array because of statmodel
effect_sizes = np.array([0.2, 0.5, 0.8])

#plot power curve using `chi_ square_power`
chi_square_power.plot_power(
    dep_var="nobs",
    nobs= n_observations,
    effect_size=effect_sizes,
    alpha=0.05,
    n_bins=2
);

## Calculate Subjects per Day

In the previous lesson, we decided that our experiment would focus on the subset of applicants who don't take the admissions quiz immediately after creating an account. We know we need around 400 observations from this subset, but how long do we need to run our experiment for in order to get that number?

To answer that question, we first need to calculate how many such applicants open an account each day.

Use the [`aggregate`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.aggregate) method to calculate how many new accounts were created each day included in the database

In [ ]:
result = ds_app.aggregate(
    [
        {"$match": {"admissionsQuiz": "incomplete"}},
        {
            "$group": {
                "_id": {"$dateTrunc": {"date": "$createdAt", "unit": "day"}},
                "count": {"$sum": 1}
            }
        }
    ]
)
print("result type:", type(result))

 Read your `result` from the previous task into the Series `no_quiz`. The Series index should be called `"date"`, and the name should be `"new_users"`.

In [ ]:
no_quiz = (
    pd.DataFrame(result)
    .rename({"_id": "date", "count": "new_users"}, axis=1)
    .set_index("date")
    .sort_index()
    .squeeze()

)

print("no_quiz type:", type(no_quiz))
print("no_quiz shape:", no_quiz.shape)
no_quiz.head()

In [ ]:
no_quiz.describe()

 Create a histogram of `no_quiz`. Be sure to label the x-axis `"New Users with No Quiz"`, the y-axis `"Frequency [count]"`, and use the title `"Distribution of Daily New Users with No Quiz"`.

In [ ]:
# Create histogram of `no_quiz`
no_quiz.hist()
# Add axis labels and title
plt.xlabel("No-Quiz Applicants")
plt.ylabel("Frequency [count]")
plt.title("Distribution of Daily New Users with No Quiz")

Calculate the mean and standard deviation of the values in `no_quiz`, and assign them to the variables `mean` and `std`, respectively.


In [ ]:
no_quiz.describe()["std"]

In [ ]:
mean = no_quiz.describe()["mean"]
std = no_quiz.describe()["std"]
print("no_quiz mean:", mean)
print("no_quiz std:", std)

The exact answers you'll get here will be a little different, but you should see a mean around 40 and a standard deviation between 7 and 8. Taking those rough numbers as a guide, how many days do we need to run the experiment to make sure we get to 400 users?

Intuitively, you might think the answer is 10 days, because $10 \cdot 40 = 400$. But we can't guarantee that we'll get 40 new users every day. Some days, there will be fewer; some days, more. So how can we estimate how many days we'll need? Statistics!

The distribution we plotted above shows how many no-quiz applicants come to the site *each day*, but we can use that mean and standard deviation to create a new distribution — one for the sum of no-quiz applicants over *several days*. Let's start with our intuition, and create a distribution for 10 days.

 Calculate the mean and standard deviation of the probability distribution for the total number of sign-ups over 10 days.

In [ ]:
days = 10 # try for day 9 , 10 and 11 
sum_mean = mean * days
sum_std = std * np.sqrt(days)
print("Mean of sum:", sum_mean)
print("Std of sum:", sum_std)

With this new distribution, we want to know what the probability is that we'll have 400 or more no-quiz applicants after 10 days. We can calculate this using the **cumulative density function** or CDF. The CDF will give us the probability of having 400 or fewer no-quiz applicants, so we'll need to subtract our result from 1.

In [ ]:
# Calculate the probability of getting 400 or more sign-ups over three days.
prob_400_or_fewer = scipy.stats.norm.cdf(
    group_size * 2,
    loc=sum_mean,
    scale=sum_std
)
prob_400_or_greater = 1 - prob_400_or_fewer

print(
    f"Probability of getting 400+ no_quiz in {days} days:",
    round(prob_400_or_greater, 3),
)
#with 11days probability of covering more than 400 observation is 100% 

In [ ]:
prob_400_or_fewer

Using the `Experiment` object created below, run your experiment for the appropriate number of days.

In [ ]:
exp = Experiment(repo=client, db="wqu-abtest", collection="ds-applicants")
exp.reset_experiment()
result = exp.run_experiment(days=days)
print("result type:", type(result))
result

# Evaluating Experiment Results

After all that work, the actual running of the experiment might seem a little anticlimactic. This is because we automated the process and are working with synthetic data. Let's look at our results?

Query `ds_app` to find all the documents that are part of the experiment. 

In [ ]:
esult = ds_app.find({"inExperiment": True})
print("results type:", type(result))

In [ ]:
df = pd.DataFrame(result).dropna()
print("df type:", type(df))
print("df shape:", df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
df[["group", "admissionsQuiz"]].head()

In [ ]:
data = pd.crosstab(
    index=df["group"],
    columns=df["admissionsQuiz"],
    normalize=False
)

print("data type:", type(data))
print("data shape:", data.shape)
data

 Create a function that returns side-by-side bar chart from `data`, showing the number of complete and incomplete quizzes for both the treatment and control groups. Be sure to label the x-axis `"Group"`, the y-axis `"Frequency [count]"`, and use the title `"Admissions Quiz Completion by Group"`.

In [ ]:
def build_contingency_bar():
    # Create side-by-side bar chart
    fig = px.bar(
        data_frame=data,
        barmode="group",
        title="Admissions Quiz Completion by Group"
    )
    # Set axis labels
    fig.update_layout(
        xaxis_title="Group",
        yaxis_title="Frequency [count]",
        legend= {"title": "Admissions Quiz"}
    )
    return fig
build_contingency_bar().show()

Without doing anything else, we can see that people who got an email actually did complete the quiz more often than people who didn't. So can we conclude that, as a general rule, applicants who receive an email are more likely to complete quiz. No, not yet. After all, the difference we see could be due to chance. 

In order to determine if this difference is more than random variation, we need to take our results, put them into a **contingency table**, and run a statistical test.

Instantiate a `Table2x2` object named `contingency_table`, using the values from the `data` you created in the previous task.

In [ ]:
contingency_table = Table2x2(data.values)

print("contingency_table type:", type(contingency_table))
contingency_table.table_orig

Calculate the fitted values for your `contigency_table`

In [ ]:
# Calculate fitted values

contingency_table.fittedvalues.round(2)

Calculate the joint probabilities under independence for your `contingency_table`.

In [ ]:
# Calculate independent joint probabilities
contingency_table.independence_probabilities.round(3)

## Conduct the Chi-Squared Test

Here's where the rubber meets the road: all the previous calculations have shown us that some of the people who got an email went on to complete the quiz, but we don't know what might be driving that effect. After all, some people might be responding to getting an email, but others might have finished the quiz whether we emailed them or not. Either way, the effect we found could just as easily be due to chance as it could be a result of something we did. The only way to find out whether the result is due to chance is to calculate *statistical significance*.

There are several ways to do this, but since the rows and columns here are unordered (nominal factors), we can do a chi-square test.

Perform a chi-square test of independence on your `contingency_table` and assign the results to `chi_square_test`.


In [ ]:
chi_square_test = contingency_table.test_nominal_association()

print("chi_square_test type:", type(chi_square_test))
print(chi_square_test)

The important part of that result is the *p-value*. We set our threshold for significance at 0.05 way back at the beginning, so, for our results to be statistically significant, the p-value needs to be less than or equal to 0.05. Our p-value is much higher than 0.05, which means that the difference we saw in our side-by-side bar graph is probably due to chance. In other words, it's noise, not signal. So we can't reject our null hypothesis. 

What does this result mean? It means there may not be any difference between the groups, or that the difference is so small that we don't have the statistical power to detect it.

Since this is a simulated experiment, we can actually increase the power by re-running the experiment for a longer time. If we ran the experiment for 60 days, we might end up with a statistically-significant result. Try it and see what happens!

However, there are two important things to keep in mind. First, just because a result is *statistically* significant doesn't mean that it's *practically* significant. A 1% increase in quiz completion may not be worth the time or resources needed to run an email campaign every day. Second, when the number of observations gets very large, any small difference is going to appear statistically significant. This increases the risk of a false positive — rejecting our null hypothesis when it's actually true.

Setting the issue of significance aside for now, there's one more calculation that can be helpful in sharing the results of an experiment: the odds ratio. In other words, how much more likely is someone in the treatment group to complete the quiz versus someone in the control group?

Calculate the odds ratio for your `contingency_table`.

In [ ]:
odds_ratio = contingency_table.oddsratio.round(3)
print("Odds ratio:", odds_ratio)

The interpretation here is that for every 1 person who doesn't complete the quiz, about 1.3 people do. Keep in mind, though, that this ratio isn't actionable in the case of our experiment because our results weren't statistically significant. 

The last thing we need to do is print all the values in our contingency table.

 Print out the summary for your `contingency_table`.


In [ ]:
summary = contingency_table.summary()
print("summary type:", type(summary))
summary